In [1]:
json_array = [
    {"Revenue Notes": "$15", "Revenue Value": 15000000},
    {"Revenue Notes": "$15", "Revenue Value": 15000000},
    {"Revenue Notes": "$17", "Revenue Value": 17000000},
    {"Revenue Notes": "20", "Revenue Value": 20000000},
    {"Revenue Notes": "20", "Revenue Value": 20000000},
    {"Revenue Notes": "25", "Revenue Value": 25000000},
    {"Revenue Notes": "30", "Revenue Value": 30000000},
    {"Revenue Notes": "40", "Revenue Value": 40000000},
    {"Revenue Notes": "$42", "Revenue Value": 42000000}
]
json_array[0]

{'Revenue Notes': '$15', 'Revenue Value': 15000000}

In [4]:
import re

def extract_arr_value(revenue_note):
    # Define the regex pattern
    pattern = r"\$?(\d+(\.\d+)?[Mm]{1,2})"

    # Search for the pattern in the revenue note
    match = re.search(pattern, revenue_note)

    if match:
        arr_value = match.group(1)
        return arr_value
    else:
        return "Unknown"

# Example usage
revenue_notes = [
    "51",
    "200",
    "$8,022",
    "227346",
    "- $5M ARR this year - EOY 2024 $6.5 - $7M ARR",
    "--2018A:  $27.5mm in revenue",
    "--2019-12A:  ?10.2mm ARR",
    "--58 LinkedIn Employees 3.28.2019",
    "--FYE 2020-04-30E:  CAD$60mm in revenue",
    "-170M revenue",
    "?125M 2023",
]

ret = {}

for note in revenue_notes:
    arr_value = extract_arr_value(note)
    ret[note] = arr_value
    # print(f"Revenue Note: {note} | ARR Value: {arr_value}")

print(ret)

{'51': 'Unknown', '200': 'Unknown', '$8,022': 'Unknown', '227346': 'Unknown', '- $5M ARR this year - EOY 2024 $6.5 - $7M ARR': '5M', '--2018A:  $27.5mm in revenue': '27.5mm', '--2019-12A:  ?10.2mm ARR': '10.2mm', '--58 LinkedIn Employees 3.28.2019': 'Unknown', '--FYE 2020-04-30E:  CAD$60mm in revenue': '60mm', '-170M revenue': '170M', '?125M 2023': '125M'}


In [8]:
import pandas as pd
import re

# Load the Excel file
file_path = './Data_Extraction.xlsx'
df = pd.read_excel(file_path)

df

,Unique ID,Revenue Notes,Revenue Value,Revenue Confidence,Growth Notes,Growth Percentage,Growth Confidence,Profitability Notes,Profitability Value,Profitability Confidence,Concentrations Notes,Yes / No,Concentration Confidence
0,0011L00002iAFxBQAW,0.25,25000000,NaN,0.3,0.3,NaN,Burning $3mm,Not Profitable,NaN,NaN,Blank,NaN
1,0010V00002DTd6KQAT,3,3000000,NaN,NaN,Blank,NaN,BE by Dec'20,Break Even,NaN,NaN,Blank,NaN
2,001d00000216enzAAA,5.6,56000000,NaN,0.7,0.7,NaN,Profitable,Profitable,NaN,Large J&J Contract (details to be received),Yes,NaN
3,001VR000007YZLuYAO,10,10000000,NaN,30-40% growth,0.3,NaN,50% EBITDA,Profitable,NaN,NaN,Blank,NaN
4,0018X000032cFKpQAM,10,10000000,NaN,35%+,0.35,NaN,burning $2mm,Not Profitable,NaN,top is 7-8% of revenue,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4638,0013w00002UIwUpAAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Top 3 customers generate ~60% of revenue,NaN,NaN
4639,0010V00002QsRw2QAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,top customer is over 20% of revenue,NaN,NaN
4640,0013w00002UGmo4AAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,very heavily concentrated,NaN,NaN
4641,001d000001f6EIMAA2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,very large concentration,NaN,NaN


In [11]:
import pandas as pd
import re

# Load the Excel file
file_path = './Data_Extraction.xlsx'
df = pd.read_excel(file_path)

# Define advanced functions to extract values from the notes
def extract_revenue(note):
    # Preprocess the note to handle edge cases
    note = note.replace('?', '').replace('CAD$', '').replace('$', '')
    
    # Compile a list of regex patterns to match different formats of revenue notes
    patterns = [
        r'(\d+(\.\d+)?)(M|MM|mm|m)',  # Matches 5M, 27.5mm, 170M, etc.
        r'(\d{1,3}(,\d{3})+)',         # Matches 8,022, 227346, etc.
    ]
    
    for pattern in patterns:
        match = re.search(pattern, note, re.IGNORECASE)
        if match:
            value = match.group(1).replace(',', '')
            return float(value) if 'm' in match.group(3).lower() else float(value) / 1_000_000
    return None

# Dummy confidence calculation
def calculate_confidence(note):
    # A simple heuristic: if note contains certain keywords, boost confidence
    keywords = ['ARR', 'revenue', '2023', 'year']
    confidence = 0.5  # Base confidence
    for keyword in keywords:
        if keyword.lower() in note.lower():
            confidence += 0.1
    return min(confidence, 1.0)  # Cap confidence at 1.0

# Apply extraction functions to the dataframe
df['Revenue Value'] = df['Revenue Notes'][:10].astype(str).apply(extract_revenue)
df['Revenue Confidence'] = df['Revenue Notes'][:10].astype(str).apply(calculate_confidence)

# Save the updated dataframe back to Excel
output_path = './Updated_Data_Extraction.xlsx'
df.to_excel(output_path, index=False)

# Display the updated dataframe to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="Updated Data Extraction", dataframe=df)

# Output path for user reference
output_path


'./Updated_Data_Extraction.xlsx'

In [13]:
import openai
import pandas as pd

# Load the Excel file
file_path = './Data_Extraction.xlsx'
df = pd.read_excel(file_path)

# Set up your OpenAI API key
openai.api_key = 'YOUR_OPENAI_API_KEY'

# Define a function to extract ARR values using GPT-4
def extract_arr_value(note):
    prompt = f"Extract the ARR value from the following note: \"{note}\". If the value is not clear, return 'unknown'. Only return the number in millions or 'unknown'."
    
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=10,
        temperature=0
    )
    
    arr_value = response.choices[0].text.strip()
    
    # Postprocess the extracted value to ensure it matches the expected format
    try:
        arr_value = float(arr_value)
    except ValueError:
        arr_value = 'unknown'
    
    return arr_value

# Apply the function to the 'Revenue Notes' column
df['Revenue Value'] = df['Revenue Notes'].apply(extract_arr_value)

# Save the updated dataframe back to Excel
output_path = '/mnt/data/Updated_Data_Extraction.xlsx'
df.to_excel(output_path, index=False)

# Display the updated dataframe to the user
import ace_tools as tools; tools.display_dataframe_to_user(name="Updated Data Extraction", dataframe=df)

# Output path for user reference
output_path


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
